In [4]:
# If don't have, install only once
# !pip install kagglehub

import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("snehangsude/audible-dataset")

df = pd.read_csv(os.path.join(path, 'audible_uncleaned.csv'))

# Check first views:
df.head()
# df.describe()
# df.info()

ConnectTimeout: HTTPSConnectionPool(host='www.kaggle.com', port=443): Max retries exceeded with url: /api/v1/datasets/view/snehangsude/audible-dataset (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f9db2c49c30>, 'Connection to www.kaggle.com timed out. (connect timeout=5)'))

In [ ]:
# Clean text data in Author and Narrator columns
# Remove Writtenby: from the author column
df['author'] = df['author'].str.replace('Writtenby:', '')
# Remove Narratedby: from the narrator column
df['narrator'] = df['narrator'].str.replace('Narratedby:', '')
df.head(2)

In [ ]:
# Clean stars table
df['ratings'] = df['stars'].str.extract(r'stars(d+)\s*')
df.head()